# Анализ работы модели предсказания качества воздуха

В работе используются:
- Модель Air Quality Prediction ([Kaggle](https://www.kaggle.com/code/hamedetezadi/air-quality-prediction)), в формате [ноутбука](https://colab.research.google.com/drive/14Tlqrmg76i7S33Gc5P5iFvjh7Ml2dXxB?usp=sharing)
- Датасет  «СОСТОЯНИЕ ЗАГРЯЗНЕНИЯ АТМОСФЕРЫ В
 ГОРОДАХ РОССИИ C 2007 ГОДА» - взят с сайта [Если быть точным](https://tochno.st/datasets/air_cities)


# 1. Скачивание датасета

In [2]:
import os

import kagglehub

source_dataset_name = "shrutibhargava94/india-air-quality-data"
source_dataset_files_path = kagglehub.dataset_download(source_dataset_name)

print(f"Скачиваем исходный датасет {source_dataset_name}")
for filename in os.listdir(source_dataset_files_path):
    source = os.path.join(source_dataset_files_path, filename)
    if os.path.isfile(source):
        with open(source, 'rb') as src:
            content = src.read()

        with open(os.path.join("./data", filename), 'w') as dst:
            dst.write(content.decode('utf-8', errors='ignore'))

print(f"Исходный датасет {source_dataset_name} помещен в папку проекта data")

Скачиваем исходный датасет shrutibhargava94/india-air-quality-data
Исходный датасет shrutibhargava94/india-air-quality-data помещен в папку проекта data


## 2. Импорт необходимых зависимостей

In [3]:
import pandas as pd
import numpy as np
import joblib as jl
import re

## 3. Обработка исходных данных

In [4]:
# Загрузка данных датасета
dataset_path = 'data/data_air_cities_100_v20231129.csv'
dataset_files_path = pd.read_csv(dataset_path, sep=';', engine='python')

# Наименования столбцов, которые должны содержать только числовые данные
columns_with_numeric_values = ['air_solid_emissions', 'air_so_emissions', 'air_no_emissions', 'air_co_emissions',
                               'air_population', 'air_repeatability']

# Функция для извлечения числовых значений
def extract_only_number(value):
    if isinstance(value, str):
        match = re.search(r'\d+(\.\d+)?', value)
        return float(match.group()) if match else np.nan

    return value

# Очистка числовых данных от текста, если он есть в столбце, где это не подразумевается
for column in columns_with_numeric_values:
    dataset_files_path[column] = dataset_files_path[column].apply(extract_only_number)

# Заполнение пустых числовых данных средним значением
dataset_files_path[columns_with_numeric_values] = dataset_files_path[columns_with_numeric_values].fillna(
    dataset_files_path[columns_with_numeric_values].mean())

# Нормализация числовых данных (мин-макс нормализация)
dataset_files_path[columns_with_numeric_values] = dataset_files_path[columns_with_numeric_values].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Обработка пропущенных значений категориальных данных модой
columns_with_categorical_values = ['region', 'city', 'air_general_level', 'air_standard_index']
dataset_files_path[columns_with_categorical_values] = dataset_files_path[columns_with_categorical_values].apply(
    lambda x: x.fillna(x.mode()[0])
)

# Кодирование категориальных данных
data_encoded = pd.get_dummies(dataset_files_path, columns=columns_with_categorical_values, drop_first=True)
print(f"Обработка исходных данных завершена")

Обработка исходных данных завершена


## 4. Подготовка существующей модели

In [9]:
# Загрузка существующей модели
model_name = "Air_Quality_Prediction.joblib"
model_path = f"model/{model_name}"
model = jl.load(model_path)

# Определяем ожидаемые признаки для модели
expected_features = model.feature_names_in_
print("Ожидаемые моделью признаки:", expected_features)
print("Текущие признаки:", data_encoded.columns.tolist())

# Подготавливаем список отсутствующих в наших данных признаков
missing_features = set(expected_features) - set(data_encoded.columns)
print("Отсутствующие признаки:", data_encoded.columns.tolist())

# Добавляем отсутствующие признаки, заполняя их нулями
for feature in missing_features:
    data_encoded[feature] = 0

# Оставляем только нужные признаки для модели
data_final = data_encoded[list(expected_features)].copy()

Ожидаемые моделью признаки: ['SOi' 'Noi' 'Rpi' 'SPMi']
Текущие признаки: ['year', 'region_oktmo', 'city_oktmo', 'air_repeatability', 'air_qcp', 'air_solid_emissions', 'air_so_emissions', 'air_no_emissions', 'air_co_emissions', 'air_population', 'air_stantions', 'region_Амурская область', 'region_Архангельская область', 'region_Астраханская область', 'region_Белгородская область', 'region_Брянская область', 'region_Владимирская область', 'region_Волгоградская область', 'region_Вологодская область', 'region_Воронежская область', 'region_Еврейская автономная область', 'region_Забайкальский край', 'region_Ивановская область', 'region_Иркутская область', 'region_Калининградская область', 'region_Калужская область', 'region_Камчатский край', 'region_Карачаево-Черкесская Республика', 'region_Кемеровская область', 'region_Кировская область', 'region_Костромская область', 'region_Краснодарский край', 'region_Красноярский край', 'region_Курганская область', 'region_Курская область', 'region_Лени

## 5 Использование модели для прогнозирования

In [10]:
# Прогнозирование
predictions = model.predict(data_final)
data_final['Predictions'] = predictions

# Сохранение обработанных данных
processed_file_path = 'data/processed_data_air_quality.csv'
data_final.to_csv(processed_file_path, index=False)
print(f"Обработанные данные сохранены в {processed_file_path}")

# Сохранение прогнозов
predictions_file_path = 'data/predictions_air_quality.csv'
data_final.to_csv(predictions_file_path, index=False)
print(f"Прогнозы сохранены в {predictions_file_path}")

Обработанные данные сохранены в data/processed_data_air_quality.csv
Прогнозы сохранены в data/predictions_air_quality.csv
